In [1]:
%%capture
!pip install transformers datasets evaluate torchmetrics

In [8]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!git lfs install
!git clone https://huggingface.co/Katerina-gopher/bart-ru-summary

In [9]:
#@title Импортируем необходимые библиотеки
import torch
from torchmetrics.text.rouge import ROUGEScore
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
from datasets import load_dataset
from tqdm import tqdm
import time

In [10]:
#@title Загружаем предобученную модель и токенизатор BART
model = BartForConditionalGeneration.from_pretrained("Katerina-gopher/bart-ru-summary")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-xsum")

In [11]:
#@title Загружаем датасет 
from datasets import load_dataset

dataset = load_dataset("Katerina-gopher/ru-summary")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/Katerina-gopher___csv/Katerina-gopher--ru-summary-41b168de469a9e8c/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['summ', 'text'],
        num_rows: 121833
    })
    validation: Dataset({
        features: ['summ', 'text'],
        num_rows: 12103
    })
    test: Dataset({
        features: ['summ', 'text'],
        num_rows: 17512
    })
})

In [13]:
ds = dataset
ds = ds.pop('train')
ds

Dataset({
    features: ['summ', 'text'],
    num_rows: 121833
})

In [14]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['summ', 'text'],
        num_rows: 12103
    })
    test: Dataset({
        features: ['summ', 'text'],
        num_rows: 17512
    })
})

In [15]:
#@title Определяем функцию для обработки датасета
def process_data(data):
  # data - это список кортежей вида (source_text, target_text)
  # source_text - это текст статьи, target_text - это текст суммарии
  # возвращаем список словарей вида {'input_ids': ..., 'attention_mask': ..., 'labels': ...}
  result = []
  for target_text, source_text in data:
    if len(result) == 5000:
      break
    # токенизируем тексты
    input_ids = tokenizer.encode(source_text, return_tensors='pt', max_length=1024)
    labels = tokenizer.encode(target_text, return_tensors='pt', max_length=128)
    # добавляем маску внимания для ненулевых токенов
    attention_mask = input_ids != tokenizer.pad_token_id
    # добавляем словарь в результат
    result.append({'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels})
  return result

In [16]:
#@title Применяем функцию к датасету
train_ds = process_data(dataset['test'])
test_ds = process_data(dataset['validation'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [17]:
#@title Определяем функцию для обучения модели на одном батче данных
def train_on_batch(batch):
  # batch - это словарь вида {'input_ids': ..., 'attention_mask': ..., 'labels': ...}
  # input_ids - это тензор размера (batch_size, max_length_source)
  # attention_mask - это тензор размера (batch_size, max_length_source)
  # labels - это тензор размера (batch_size, max_length_target)
  # возвращаем значение функции потерь на батче
  # переводим модель в режим обучения
  model.train()
  # переносим данные на устройство (cpu или gpu)
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  batch = {k: v.to(device) for k, v in batch.items()}
  # делаем прямой проход модели и получаем логиты
  outputs = model(**batch)
  logits = outputs.logits
  # вычисляем значение функции потерь
  loss = outputs.loss
  # делаем обратный проход и обновляем параметры модели
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()
  # Прибавляем потерю к суммарной потере за эпоху
  # epoch_loss += loss.item()
  # Генерируем резюме из выходов модели
  labels = batch['labels']
  summaries = tokenizer.batch_decode(torch.argmax(outputs.logits, dim=-1), skip_special_tokens=True)
  # Переводим метки в текстовый формат
  references = tokenizer.batch_decode(labels, skip_special_tokens=True)
  # Обновляем метрику rouge на пакете
  return loss, summaries, references

In [18]:
#@title Определяем функцию для оценки модели на одном батче данных
def evaluate_on_batch(batch):
  # batch - это словарь вида {'input_ids': ..., 'attention_mask': ..., 'labels': ...}
  # input_ids - это тензор размера (batch_size, max_length_source)
  # attention_mask - это тензор размера (batch_size, max_length_source)
  # labels - это тензор размера (batch_size, max_length_target)
  # возвращаем список сгенерированных суммарий и значение функции потерь на батче
  # переводим модель в режим оценки
  model.eval()
  # переносим данные на устройство (cpu или gpu)
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  batch = {k: v.to(device) for k, v in batch.items()}
  # делаем прямой проход модели и получаем логиты
  outputs = model(**batch)
  logits = outputs.logits
  # вычисляем значение функции потерь
  loss = outputs.loss
  # генерируем суммарии с помощью жадного декодирования
  summaries_ids = torch.argmax(logits, dim=-1)
  # преобразуем тензоры в строки
  summaries = tokenizer.batch_decode(summaries_ids, skip_special_tokens=True)
  # возвращаем список суммарий и значение функции потерь
  return summaries, loss.item()

In [19]:
#@title Определяем функцию потерь, оптимизатор и скорость обучения для модели
criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=1)

In [20]:
rouge = ROUGEScore(rouge_keys=('rouge1', 'rougeL', 'rougeLsum')) 

In [21]:
#@title Задаем лучшее значение метрики rougeRus и имя файла для сохранения модели
best_rougeL = 0
model_file = 'bart-ru-summary.pt'

In [22]:
#@title Определяем количество эпох обучения
epochs = 5

In [ ]:
# Импортируем модуль pandas
import pandas as pd

print("Начало обучения")
# for epoch in tqdm(range(epochs), ascii="░▒█"):
# Обнуляем суммарный rouge за эпоху
# epoch_rouge_header = ['rouge1.precision','rouge1.recall', 'rouge1.fmeasure',
#                       'rouge2.precision', 'rouge2.recall', 'rouge2.fmeasure',
#                       'rougeL.precision', 'rougeL.recall', 'rougeL.fmeasure']
# df = pd.DataFrame(epoch_rouge, index=[0])
# df.to_csv("epoch_rouge.csv")
for epoch in range(epochs):
  # Обнуляем суммарную потерю за эпоху
  epoch_loss = 0.0
  epoch_loss1 = 0.0
  # Обнуляем суммарный rouge за эпоху
  epoch_rouge = {'rouge1': {'precision': 0.0, 'recall': 0.0, 'fmeasure': 0.0},
                 'rouge2': {'precision': 0.0, 'recall': 0.0, 'fmeasure': 0.0},
                 'rougeL': {'precision': 0.0, 'recall': 0.0, 'fmeasure': 0.0}}
  # Перебираем пакеты данных
  print("Обучения в батче")
  for batch in tqdm(test_ds, ascii="░▒█"):
    # переводим модель в режим обучения
    model.train()
    # переносим данные на устройство (cpu или gpu)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    batch = {k: v.to(device) for k, v in batch.items()}
    # делаем прямой проход модели и получаем логиты
    outputs = model(**batch)
    logits = outputs.logits
    # вычисляем значение функции потерь
    loss = outputs.loss
    # print(f"{i}: {loss.item()}")
    # делаем обратный проход и обновляем параметры модели
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    # Прибавляем потерю к суммарной потере за эпоху
    epoch_loss += loss.item()
  # Выводим среднюю потерю за эпоху
  print(f'\nEpoch {epoch + 1}\nLoss train: {epoch_loss / len(test_ds)}')
  # оценка
  print("Тест в батче")
  for batch in tqdm(test_ds, ascii="░▒█"):
    # переводим модель в режим оценки
    model.eval()
    # переносим данные на устройство (cpu или gpu)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    batch = {k: v.to(device) for k, v in batch.items()}
    # делаем прямой проход модели и получаем логиты
    outputs = model(**batch)
    logits = outputs.logits
    # вычисляем значение функции потерь
    loss = outputs.loss
    # Прибавляем потерю к суммарной потере за эпоху
    epoch_loss1 += loss.item()
    # Генерируем резюме из выходов модели
    labels = batch['labels']
    summaries = tokenizer.batch_decode(torch.argmax(outputs.logits, dim=-1), skip_special_tokens=True)
    # Переводим метки в текстовый формат
    references = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Обновляем метрику rouge на пакете
    rouge.update(summaries, references)
  # Выводим среднюю потерю за эпоху
  print(f'\nEpoch {epoch + 1}\nLoss test: {epoch_loss1 / len(test_ds)}')
  # Выводим результаты на экран
  # Вычисляем и выводим средний rouge за эпоху
  epoch_rouge = rouge.compute()
  print(epoch_rouge)
  print(f'ROUGE-1: {epoch_rouge["rouge1_fmeasure"]:.4f}')
  print(f'ROUGE-L: {epoch_rouge["rougeL_fmeasure"]:.4f}')
  print(f'ROUGE-Lsum: {epoch_rouge["rougeLsum_fmeasure"]:.4f}')
  # Обновляем скорость обучения в зависимости от метрики rougeL
  scheduler.step(epoch_rouge['rougeL_fmeasure'])
  # Если метрика rougeL улучшилась, сохраняем модель
  if epoch_rouge['rougeL_fmeasure'] > best_rougeL:
    best_rougeL = epoch_rouge['rougeL_fmeasure']
    torch.save(model.state_dict(), model_file)
    print('Model saved!')
  df = pd.DataFrame(epoch_rouge, index=[epoch + 1])
  # Выводим датафрейм в табличном виде
  print(df)
  # Выводим датафрейм в файл формата CSV
  df.to_csv("epoch_rouge.csv", mode='a', header=False)

Начало обучения
Обучения в батче


  4%|░░░░░░░░░░| 219/5000 [19:19<7:19:22,  5.51s/it]

In [ ]:
model.save_pretrained("bart-ru-small")

арпаг